In [111]:
from google.cloud import storage
import pandas as pd ## for dataset and eda
import numpy as np ## for eda
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm


# %matplotlib inline
print("All good")

All good


In [112]:
# Importing the libraries needed
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer # do not forget to conda install transformers
print("All packages are imported")

All packages are imported


In [113]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(f"GPU usage is set up: {device}")

GPU usage is set up: cuda


In [114]:
cuda.is_available()

True

In [5]:
bucket_name = "firstprojectdl"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

print(bucket)
print('Great, we now have access to our first bucket on google cloud storage where we put our data')

<Bucket: firstprojectdl>
Great, we now have access to our first bucket on google cloud storage where we put our data


In [54]:
from google.cloud import storage
import pandas as pd

bucket_name = "firstprojectdl"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

# When you have your files in a subfolder of the bucket.
my_prefix = "data/hw2/" # the name of the subfolder
blobs = bucket.list_blobs(prefix = my_prefix, delimiter = '/')


for blob in blobs:
    print(blob.name)
    if(blob.name != my_prefix): # ignoring the subfolder itself 
        file_name = blob.name.replace(my_prefix, "")
        blob.download_to_filename(file_name) # download the file to the machine
        print(file_name)
print(f"we imported the {file_name} successfully")

data/hw2/
data/hw2/nodeid2paperid.csv
nodeid2paperid.csv
data/hw2/sample.csv
sample.csv
data/hw2/testData.csv
testData.csv
data/hw2/textData.csv
textData.csv
data/hw2/trainData.csv
trainData.csv
we imported the trainData.csv successfully


In [55]:
nodeid2paperid = pd.read_csv('nodeid2paperid.csv')
nodeid2paperid.head()
nodeid2paperid.rename(columns={'node idx': 'id'}, inplace=True)
nodeid2paperid.head()

,id,paper id
0,104447,630234
1,15858,803423
2,107156,1102481
3,82077,1810480
4,42436,2131697


In [56]:
test = pd.read_csv('testData.csv')
test.head()

,id
0,137832
1,137833
2,137834
3,137836
4,137837


In [57]:
test.shape

(13718, 1)

In [58]:
text = pd.read_csv('textData.csv')
text.head()

,paper id,title,abstract
0,630234,spreadsheets on the move an evaluation of mobi...,The power of mobile devices has increased dram...
1,803423,multi view metric learning for multi view vide...,Traditional methods on video summarization are...
2,1102481,big data analytics in future internet of things,Current research on Internet of Things (IoT) m...
3,1810480,cryptographic hardening of d sequences,This paper shows how a one-way mapping using m...
4,2131697,gesture based continuous authentication for we...,We study the feasibility of touch gesture beha...


In [59]:
text.shape

(73718, 3)

In [60]:
train = pd.read_csv('trainData.csv')
train.head()

,label,id
0,4,0
1,5,1
2,8,3
3,6,6
4,4,7


In [61]:
train.shape

(60000, 2)

In [62]:
sample = pd.read_csv('sample.csv')
sample.head()

,id,label
0,137832,0
1,137833,0
2,137834,0
3,137836,0
4,137837,0


In [63]:
sample.shape

(13718, 2)

In [64]:
## Let us merge text in train and test
# train
trainData = pd.merge(train, nodeid2paperid, on='id', how='inner')

In [65]:
trainData = pd.merge(trainData, text, on ='paper id', how='inner')

In [66]:
trainData.head()

,label,id,paper id,title,abstract
0,4,0,9657784,evasion attacks against machine learning at te...,"In security-sensitive applications, the succes..."
1,5,1,39886162,how hard is computing parity with noisy commun...,We show a tight lower bound of $\Omega(N \log\...
2,8,3,121432379,a promise theory perspective on data networks,Networking is undergoing a transformation thro...
3,6,6,1444859417,webvrgis based city bigdata 3d visualization a...,This paper shows the WEBVRGIS platform overlyi...
4,4,7,1483430697,information theoretic authentication and secre...,"In the splitting model, information theoretic ..."


In [67]:
trainData.shape

(60000, 5)

In [68]:
# test
testData = pd.merge(test, nodeid2paperid, on='id', how='inner')

In [69]:
testData.head()

,id,paper id
0,137832,2403725649
1,137833,2404740077
2,137834,2407125866
3,137836,2408327416
4,137837,2412021890


In [70]:
testData = pd.merge(testData, text, on ='paper id', how='inner')

In [71]:
testData.head()

,id,paper id,title,abstract
0,137832,2403725649,patchlift fast and exact computation of patch ...,"In this paper, we propose a fast algorithm cal..."
1,137833,2404740077,the unreasonable effectiveness of address clus...,Address clustering tries to construct the one-...
2,137834,2407125866,end to end goal driven web navigation,We propose a goal-driven web navigation as a b...
3,137836,2408327416,complexity measures for map reduce and compari...,The programming paradigm Map-Reduce and its ma...
4,137837,2412021890,a parallel implementation of the ensemble kalm...,This paper discusses an efficient parallel imp...


In [72]:
testData.shape

(13718, 4)

In [73]:
# # Removing unwanted columns and only leaving title and abstract of the scientific article and the category which will be the label
trainData = trainData[['id','title', 'abstract', 'label']]
labelListData = list(trainData['label'].unique())
trainData.head()

,id,title,abstract,label
0,0,evasion attacks against machine learning at te...,"In security-sensitive applications, the succes...",4
1,1,how hard is computing parity with noisy commun...,We show a tight lower bound of $\Omega(N \log\...,5
2,3,a promise theory perspective on data networks,Networking is undergoing a transformation thro...,8
3,6,webvrgis based city bigdata 3d visualization a...,This paper shows the WEBVRGIS platform overlyi...,6
4,7,information theoretic authentication and secre...,"In the splitting model, information theoretic ...",4


In [74]:
testData = testData[['id','title', 'abstract']]
testData.head()

,id,title,abstract
0,137832,patchlift fast and exact computation of patch ...,"In this paper, we propose a fast algorithm cal..."
1,137833,the unreasonable effectiveness of address clus...,Address clustering tries to construct the one-...
2,137834,end to end goal driven web navigation,We propose a goal-driven web navigation as a b...
3,137836,complexity measures for map reduce and compari...,The programming paradigm Map-Reduce and its ma...
4,137837,a parallel implementation of the ensemble kalm...,This paper discusses an efficient parallel imp...


In [75]:
## merge the title and abstract columns together 
# trainData['description'] = trainData['title'].str.cat(trainData['abstract'],sep=" ")
trainData['description'] = trainData['title']

# Remove stop words in description and title ## Actually mention why you may not want to remove stop words: https://stackoverflow.com/questions/63633534/is-it-necessary-to-do-stopwords-removal-stemming-lemmatization-for-text-classif

In [76]:
trainData = trainData[['description', 'label']]

In [77]:
trainData.isna().sum()

description    0
label          0
dtype: int64

In [78]:
len(labelListData) # 20 labels and not 40

20

In [79]:
labelListData

[4, 5, 8, 6, 3, 16, 19, 14, 10, 0, 2, 18, 9, 13, 11, 1, 7, 15, 17, 12]

In [80]:
# biggest text in column
trainData['description'].apply(lambda x: len(x)).max()

255

In [81]:
trainData

,description,label
0,evasion attacks against machine learning at te...,4
1,how hard is computing parity with noisy commun...,5
2,a promise theory perspective on data networks,8
3,webvrgis based city bigdata 3d visualization a...,6
4,information theoretic authentication and secre...,4
...,...,...
59995,incentivizing users of data centers participat...,5
59996,semantic change detection with hypermaps,16
59997,computing with polynomial ordinary differentia...,9
59998,on energy efficiency in wireless networks a ga...,8


In [82]:
## testData['description'] = testData['title'].str.cat(testData['abstract'],sep=" ")
testData['description'] = testData['title']

In [83]:
testData

,id,title,abstract,description
0,137832,patchlift fast and exact computation of patch ...,"In this paper, we propose a fast algorithm cal...",patchlift fast and exact computation of patch ...
1,137833,the unreasonable effectiveness of address clus...,Address clustering tries to construct the one-...,the unreasonable effectiveness of address clus...
2,137834,end to end goal driven web navigation,We propose a goal-driven web navigation as a b...,end to end goal driven web navigation
3,137836,complexity measures for map reduce and compari...,The programming paradigm Map-Reduce and its ma...,complexity measures for map reduce and compari...
4,137837,a parallel implementation of the ensemble kalm...,This paper discusses an efficient parallel imp...,a parallel implementation of the ensemble kalm...
...,...,...,...,...
13713,169336,confidence guided stereo 3d object detection w...,Accurate and reliable 3D object detection is v...,confidence guided stereo 3d object detection w...
13714,169338,sentinet detecting localized universal attacks...,SentiNet is a novel detection framework for lo...,sentinet detecting localized universal attacks...
13715,169340,learning compositional rules via neural progra...,"Many aspects of human reasoning, including lan...",learning compositional rules via neural progra...
13716,169341,certified defenses for adversarial patches,Adversarial patch attacks are among one of the...,certified defenses for adversarial patches


In [84]:
testData = testData[['description']]

In [37]:
testData

,description
0,patchlift fast and exact computation of patch ...
1,the unreasonable effectiveness of address clus...
2,end to end goal driven web navigation
3,complexity measures for map reduce and compari...
4,a parallel implementation of the ensemble kalm...
...,...
13713,confidence guided stereo 3d object detection w...
13714,sentinet detecting localized universal attacks...
13715,learning compositional rules via neural progra...
13716,certified defenses for adversarial patches


In [86]:
# biggest text in column
testData['description'].apply(lambda x: len(x)).max()

187

In [87]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 5
LEARNING_RATE = 2e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)

In [40]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        description = str(self.data.description[index])
        description = " ".join(description.split())
        inputs = self.tokenizer.encode_plus(
            description,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.label[index], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [88]:
# Creating the dataset and dataloader for the neural network
train_size = 0.8
train_dataset= trainData.sample(frac=train_size,random_state=200)
val_dataset= trainData.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(train.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("VALIDATION Dataset: {}".format(val_dataset.shape))

training_set = Triage(train_dataset, tokenizer, MAX_LEN)
validation_set = Triage(val_dataset, tokenizer, MAX_LEN)

FULL Dataset: (60000, 2)
TRAIN Dataset: (48000, 2)
VALIDATION Dataset: (12000, 2)


In [89]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

val_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
validation_loader = DataLoader(validation_set, **val_params)

In [43]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 20)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [44]:
model = DistilBERTClass()
model.to(device)

DistilBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_featu

In [46]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)

In [47]:
# Function to calcuate the accuracy of the model

def calculate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [48]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calculate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%1000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 1000 steps: {loss_step}")
            print(f"Training Accuracy per 1000 steps: {accu_step}")
        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [49]:
for epoch in tqdm(range(EPOCHS)):
    train(epoch)

  0%|          | 0/5 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length t

Training Loss per 1000 steps: 3.043318748474121
Training Accuracy per 1000 steps: 6.25
Training Loss per 1000 steps: 1.2744178297398212
Training Accuracy per 1000 steps: 64.16396103896103


 20%|██        | 1/5 [17:38<1:10:32, 1058.24s/it]

The Total Accuracy for Epoch 0: 66.59166666666667
Training Loss Epoch: 1.1726985195875168
Training Accuracy Epoch: 66.59166666666667


/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

...

The Total Accuracy for Epoch 3: 86.75625
Training Loss Epoch: 0.43820720253884793
Training Accuracy Epoch: 86.75625


/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

Training Loss per 1000 steps: 0.4278838634490967
Training Accuracy per 1000 steps: 87.5
Training Loss per 1000 steps: 0.3162710902082932
Training Accuracy per 1000 steps: 90.57192807192807


100%|██████████| 5/5 [1:29:17<00:00, 1071.52s/it]

The Total Accuracy for Epoch 4: 90.35833333333333
Training Loss Epoch: 0.32167828488349914
Training Accuracy Epoch: 90.35833333333333


In [50]:
def valid(epoch):
    model.eval()
    tr_loss = 0
    nb_tr_steps = 0
    n_correct = 0 
    nb_tr_examples = 0
    with torch.no_grad():
        for _, data in enumerate(validation_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calculate_accu(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%1000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 1000 steps: {loss_step}")
                print(f"Validation Accuracy per 1000 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu


In [90]:
for epoch in tqdm(range(EPOCHS)):
    acc = valid(epoch)
    print("Accuracy on test data = %0.2f%%" % acc)

  0%|          | 0/5 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Validation Loss per 1000 steps: 0.7389875054359436
Validation Accuracy per 1000 steps: 81.25


 20%|██        | 1/5 [01:35<06:20, 95.04s/it]

Validation Loss Epoch: 0.8925721027056376
Validation Accuracy Epoch: 75.81666666666666
Accuracy on test data = 75.82%
Validation Loss per 1000 steps: 1.2503362894058228
Validation Accuracy per 1000 steps: 68.75


 40%|████      | 2/5 [03:09<04:44, 94.95s/it]

...

Validation Loss Epoch: 0.8925721114873886
Validation Accuracy Epoch: 75.81666666666666
Accuracy on test data = 75.82%
Validation Loss per 1000 steps: 0.7300519943237305
Validation Accuracy per 1000 steps: 75.0


 80%|████████  | 4/5 [06:19<01:34, 94.96s/it]

Validation Loss Epoch: 0.8925721105734508
Validation Accuracy Epoch: 75.81666666666666
Accuracy on test data = 75.82%
Validation Loss per 1000 steps: 0.9180948138237
Validation Accuracy per 1000 steps: 78.125


100%|██████████| 5/5 [07:54<00:00, 94.90s/it]

Validation Loss Epoch: 0.8925721062421799
Validation Accuracy Epoch: 75.81666666666666
Accuracy on test data = 75.82%


In [103]:
class TriageTest(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        description = str(self.data.description[index])
        description = " ".join(description.split())
        inputs = self.tokenizer.encode_plus(
            description,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
        } 
    
    def __len__(self):
        return self.len

In [93]:
testSet = TriageTest(testData, tokenizer, MAX_LEN)

In [104]:
test_params = {'batch_size': 32,
                'shuffle': False,
                'num_workers': 0
                }


testing_loader = DataLoader(testSet, **test_params)

In [105]:
predictions = []
model.eval()
with torch.no_grad():
    for _, data in tqdm(enumerate(testing_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        outputs = model(ids, mask).squeeze()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        predictions.append(big_idx)



429it [01:38,  4.37it/s]


In [106]:
print(predictions)
listOfList = [list(x.cpu().numpy()) for x in predictions]
result = [item for sublist in listOfList for item in sublist]

[tensor([16,  8, 10,  5, 19, 15,  4,  5,  8,  8, 16, 10,  9,  0, 16, 10, 16, 16,
         2, 16, 16, 16, 19, 19, 11,  8, 18, 16, 16, 10, 16, 16],
       device='cuda:0'), tensor([10, 16, 16, 16,  8, 19,  2, 19, 16, 16,  8,  8, 14, 16,  8, 16,  6,  7,
         1, 16, 16,  4, 13,  9, 19,  8,  4,  3, 16, 16,  5, 16],
       device='cuda:0'), tensor([16,  4,  5,  2, 16, 10,  2, 16,  2, 19,  7,  4,  4, 10, 11, 16, 10, 16,
        19, 16, 16,  9,  4, 16, 19, 16,  2, 13,  3,  8, 16,  5],
       device='cuda:0'), tensor([16,  8,  7,  4, 13, 16,  8, 16, 16, 10,  4,  2, 16,  4, 10, 16,  2, 16,
        10, 16, 16, 10, 16,  8,  2, 16, 16, 16, 16, 16, 16,  5],
       device='cuda:0'), tensor([ 6, 16, 13,  5,  4,  0,  5, 16,  8,  3, 16,  5, 16, 16, 15, 16,  2, 10,
        16, 16, 16, 10, 16, 13,  0, 18, 16,  4,  2, 10, 10, 16],
       device='cuda:0'), tensor([ 8, 16, 16, 16, 16, 16, 16, 16, 16,  5,  8,  8,  8, 15,  3, 16,  5, 16,
         3, 16, 16, 13, 16, 10, 16, 17, 16, 16, 16,  3, 16,  8],
    

In [107]:
len(result)
result

[16,
 8,
 10,
 5,
 19,
 15,
 4,
 5,
 8,
 8,
 16,
 10,
 9,
 0,
 16,
 10,
 16,
 16,
 2,
 16,
 16,
 16,
 19,
 19,
 11,
 8,
 18,
 16,
 16,
 10,
 16,
 16,
 10,
 16,
 16,
 16,
 8,
 19,
 2,
 19,
 16,
 16,
 8,
 8,
 14,
 16,
 8,
 16,
 6,
 7,
 1,
 16,
 16,
 4,
 13,
 9,
 19,
 8,
 4,
 3,
 16,
 16,
 5,
 16,
 16,
 4,
 5,
 2,
 16,
 10,
 2,
 16,
 2,
 19,
 7,
 4,
 4,
 10,
 11,
 16,
 10,
 16,
 19,
 16,
 16,
 9,
 4,
 16,
 19,
 16,
 2,
 13,
 3,
 8,
 16,
 5,
 16,
 8,
 7,
 4,
 13,
 16,
 8,
 16,
 16,
 10,
 4,
 2,
 16,
 4,
 10,
 16,
 2,
 16,
 10,
 16,
 16,
 10,
 16,
 8,
 2,
 16,
 16,
 16,
 16,
 16,
 16,
 5,
 6,
 16,
 13,
 5,
 4,
 0,
 5,
 16,
 8,
 3,
 16,
 5,
 16,
 16,
 15,
 16,
 2,
 10,
 16,
 16,
 16,
 10,
 16,
 13,
 0,
 18,
 16,
 4,
 2,
 10,
 10,
 16,
 8,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 5,
 8,
 8,
 8,
 15,
 3,
 16,
 5,
 16,
 3,
 16,
 16,
 13,
 16,
 10,
 16,
 17,
 16,
 16,
 16,
 3,
 16,
 8,
 4,
 16,
 16,
 16,
 19,
 16,
 2,
 16,
 16,
 10,
 16,
 16,
 8,
 16,
 4,
 19,
 16,
 16,
 4,
 16,
 4,
 8,
 9,
 16,

In [108]:
submission = sample
submission['label'] = result

In [109]:
submission

,id,label
0,137832,16
1,137833,8
2,137834,10
3,137836,5
4,137837,19
...,...,...
13713,169336,16
13714,169338,4
13715,169340,10
13716,169341,4


In [110]:
submission.to_csv('submission.csv', index = False)

In [ ]:
## list of things left to experiment and fine tune
# 1. train bert with abstract only
# 2. train bert with title and abstract
# 3. train bert with batch size at 16 or 32
# 4. Tweak the max length for padding depending on the distribution length of the paragraphs
## 5. Very important: also try with 4 epochs
## Play with the drop out level at 0.2, 0.3, and 0.5